In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q keras
import keras

In [ ]:
! unzip '/content/drive/MyDrive/Copy of Dataset for CNN.zip'

In [2]:
import tensorflow as tf

In [7]:
import os
import random
import shutil

# Define the paths to the source folder and destination folders
source_folder = '/content/PlantVillage/Potato___Late_blight'
destination_folders = ['/content/images/train/Potato___Late_blight', '/content/images/test/Potato___Late_blight', '/content/images/eval/Potato___Late_blight']

# Define the fractions of images for each destination folder
fractions = [0.7, 0.15, 0.15]  # 70% to train, 15% to validation, 15% to test

# Create destination folders if they don't exist
for folder in destination_folders:
    os.makedirs(folder, exist_ok=True)

# Get list of image files in source folder
image_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

# Calculate the number of images to move to each destination folder
total_images = len(image_files)
num_images_per_folder = [int(total_images * fraction) for fraction in fractions]

# Shuffle the image files
random.shuffle(image_files)

# Move images to destination folders
start_index = 0
for i, folder in enumerate(destination_folders):
    end_index = start_index + num_images_per_folder[i]
    for image in image_files[start_index:end_index]:
        src_path = os.path.join(source_folder, image)
        dst_path = os.path.join(folder, image)
        shutil.move(src_path, dst_path)
    start_index = end_index

print("Images distributed successfully.")

Images distributed successfully.


In [ ]:
model = tf.keras.models.Sequential([

    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    # tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),

    tf.keras.layers.Dense(3, activation='softmax')
])

In [4]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 740,355 (2.82 MB)

 Trainable params: 740,355 (2.82 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,               # Rescale pixel values to [0, 1]
      horizontal_flip=True,         # Enable horizontal flipping
      vertical_flip=True,           # Enable vertical flipping
      rotation_range=30,           # Randomly rotate images up to 30 degrees
      fill_mode='nearest',
      )

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/images/train',  # This is the source directory for training images
        target_size=(256, 256),  # All images will be resized to 256x256
        batch_size=32,
        class_mode='categorical')

validation_datagen = ImageDataGenerator(
        rescale=1./255,
        # horizontal_flip=True,         # Enable horizontal flipping
        # vertical_flip=True,           # Enable vertical flipping
        # rotation_range=50,           # Randomly rotate images up to 50 degrees
        # fill_mode='nearest',
        )

validation_generator = validation_datagen.flow_from_directory(
        '/content/images/eval',
        target_size=(256,256),
        class_mode='categorical')


Found 1506 images belonging to 3 classes.
Found 322 images belonging to 3 classes.


In [26]:
# class StopTrainingOnAccuracy(tf.keras.callbacks.Callback):
#     def __init__(self, target_accuracy=0.998):
#         super(StopTrainingOnAccuracy, self).__init__()
#         self.target_accuracy = target_accuracy

#     def on_epoch_end(self, epoch, logs=None):
#         if logs['val_accuracy'] >= self.target_accuracy:
#             print(f"\nReached validation accuracy of {self.target_accuracy}! Stopping training.")
#             self.model.stop_training = True


In [12]:
# stop_training_callback = StopTrainingOnAccuracy(target_accuracy=0.998)

In [27]:
history = model.fit(
      train_generator,
      epochs=50,
      verbose=1,
      validation_data=validation_generator)

Epoch 1/50
48/48 [==============================] - 61s 513ms/step - loss: 0.9378 - acc: 0.4807 - val_loss: 0.8862 - val_acc: 0.4658
Epoch 2/50
48/48 [==============================] - 25s 521ms/step - loss: 0.7637 - acc: 0.6660 - val_loss: 0.6063 - val_acc: 0.7733
Epoch 3/50
48/48 [==============================] - 24s 508ms/step - loss: 0.4415 - acc: 0.8400 - val_loss: 0.4009 - val_acc: 0.8758
Epoch 4/50
48/48 [==============================] - 25s 505ms/step - loss: 0.3220 - acc: 0.8785 - val_loss: 0.3064 - val_acc: 0.8696
Epoch 5/50
48/48 [==============================] - 25s 520ms/step - loss: 0.2782 - acc: 0.8785 - val_loss: 0.3447 - val_acc: 0.8354
Epoch 6/50
48/48 [==============================] - 24s 509ms/step - loss: 0.2137 - acc: 0.9183 - val_loss: 0.2144 - val_acc: 0.8851
Epoch 7/50
48/48 [==============================] - 24s 495ms/step - loss: 0.1962 - acc: 0.9250 - val_loss: 0.2741 - val_acc: 0.9099
Epoch 8/50
48/48 [==============================] - 27s 547ms/step - 

In [28]:
model_path='/content/model'
model.save(model_path)

print("Model saved successfully at:", model_path)

Model saved successfully at: /content/model


In [29]:
test_datagen = ImageDataGenerator(
    rescale=1./255  # Rescale pixel values to [0, 1]
)

test_generator = test_datagen.flow_from_directory(
    '/content/images/test',  # Path to your test data directory
    target_size=(256, 256),  # All images will be resized to 256x256
    batch_size=32,
    class_mode='categorical'
)

Found 322 images belonging to 3 classes.


In [31]:
model_path = '/content/model'  # Update with the path to your saved model
model = tf.keras.models.load_model(model_path)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

11/11 [==============================] - 1s 65ms/step - loss: 0.0692 - acc: 0.9814
Test Loss: 0.06923475861549377
Test Accuracy: 0.9813664555549622


In [34]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model_path = '/content/model'  # Update with the path to your saved model
model = tf.keras.models.load_model(model_path)

# Load the image
img_path = '/content/images/potato-blight.jpg'  # Update with the path to your image
img = image.load_img(img_path, target_size=(256, 256))  # Assuming you resize your images to 256x256 during training
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Preprocess the image (same preprocessing as training data)
img_array /= 255.0  # Normalize pixel values

# Perform prediction
predictions = model.predict(img_array)

# Get the predicted class label
predicted_class = np.argmax(predictions[0])

# Print the predicted class
print(f'Predicted class index: {predicted_class}')

1/1 [==============================] - 0s 88ms/step
Predicted class index: 0


In [35]:
# Class indices mapping for training generator
train_class_indices = train_generator.class_indices
print("Class indices for training generator:", train_class_indices)

# Class indices mapping for validation generator
validation_class_indices = validation_generator.class_indices
print("Class indices for validation generator:", validation_class_indices)

# Invert the dictionary to map indices to class labels
train_label_indices = {v: k for k, v in train_class_indices.items()}
validation_label_indices = {v: k for k, v in validation_class_indices.items()}

# Print class labels corresponding to indices
print("Class labels for training generator:", train_label_indices)
print("Class labels for validation generator:", validation_label_indices)

Class indices for training generator: {'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}
Class indices for validation generator: {'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}
Class labels for training generator: {0: 'Potato___Early_blight', 1: 'Potato___Late_blight', 2: 'Potato___healthy'}
Class labels for validation generator: {0: 'Potato___Early_blight', 1: 'Potato___Late_blight', 2: 'Potato___healthy'}
